In [10]:
"""
Implementation for the prime paths generation from:
https://github.com/heshenghuan/Prime-Path-Coverage
"""

import codecs as cs


def readGraphFromFile(src):
    """Read a graph structure from given file."""
    with cs.open(src, 'r', 'utf-8') as graphFile:
        # Read nodes set of graph
        graphFile.readline()
        nodes = [int(n) for n in graphFile.readline().split()]
        # Read initial nodes set of graph
        graphFile.readline()
        initNodes = [int(n) for n in graphFile.readline().split()]
        # Read end nodes set of graph
        graphFile.readline()
        endNodes = [int(n) for n in graphFile.readline().split()]
        # Read edges set of graph
        graphFile.readline()
        edges = {}
        for i in nodes:
            s = graphFile.readline().strip().split()
            if len(s) >= 1:
                edges[i] = [int(n) for n in s if n != '-1']
            else:
                edges[i] = []
        graph = {'nodes': nodes, 'init': initNodes,
                 'end': endNodes, 'edges': edges}
        return graph



def isPrimePath(path, graph):
    """Whether a path is a prime path."""
    if len(path) >= 2 and path[0] == path[-1]:
        return True
    elif reachHead(path, graph) and reachEnd(path, graph):
        return True
    else:
        return False


def reachHead(path, graph):
    """
    Whether the path can be extended at head, and the extended path is still
    a simple path.
    """
    former_nodes = filter(lambda n: path[0] in graph[
                          'edges'][n], graph['nodes'])
    for n in former_nodes:
        if n not in path or n == path[-1]:
            return False
    return True


def reachEnd(path, graph):
    """
    Whether the path can be extended at tail, and the extended path is still
    a simple path.
    """
    later_nodes = graph['edges'][path[-1]]
    for n in later_nodes:
        if n not in path or n == path[0]:
            return False
    return True


def extendable(path, graph):
    """Whether a path is extendable."""
    if isPrimePath(path, graph) or reachEnd(path, graph):
        return False
    else:
        return True


def findSimplePath(graph, exPaths, paths=[]):
    """Find the simple paths of a graph."""
    paths.extend(filter(lambda p: isPrimePath(p, graph), exPaths))
    exPaths = filter(lambda p: extendable(p, graph), exPaths)
    newExPaths = []
    for p in exPaths:
        for nx in graph['edges'][p[-1]]:
            if nx not in p or nx == p[0]:
                newExPaths.append(p + (nx, ))
    if len(newExPaths) > 0:
        findSimplePath(graph, newExPaths, paths)


def findPrimePaths(graph):
    """Find the prime paths of a graph."""
    exPaths = [(n, ) for n in graph['nodes']]
    simplePaths = []
    # recursively finding the simple paths of the graph
    findSimplePath(graph, exPaths, simplePaths)
    primePaths = sorted(simplePaths, key=lambda a: (len(a), a))

    return primePaths

def rotate_cycle(cycle):
    """
    Given a cycle like [A, B, C, D, A],
    return all its rotations:
    [A, B, C, D, A], [B, C, D, A, B], ...
    """
    base = cycle[:-1]
    rotations = []
    n = len(base)
    for i in range(n):
        rotated = base[i:] + base[:i] + [base[i]]
        rotations.append(rotated)
    return rotations

def get_graph(digraph, number_of_nodes):
    graph = {}
    for key, value in dict(digraph("B0"))["B0"].items():
        graph_key = key
        if key == "S":
            graph_key = 0
        if key == "T":
            graph_key = number_of_nodes + 1

        values = []
        for val in value:
            if val == "S":
                values.append(0)
            elif val == "T":
                values.append(number_of_nodes + 1)
            else:
                values.append(int(val))
        graph[int(graph_key)] = values
        

    return {'nodes': list(graph.keys()), 'edges': graph}
    

In [12]:
import time
import random
from diblob.algorithms import TarjanSSC, PrimePathsGenerator
from diblob.generators import  RandomSESEPathBased


In [ ]:
NUMBER_OF_EXPERIMENTS = 5
MAX_PATH_LEN = 20
MIN_PATH_LEN = 4
MIN_NUMBER_OF_PATHS = 2
MAX_NUMBER_OF_PATHS = 8
MIN_NUMBER_OF_NODES = 10
MAX_NUMBER_OF_NODES = 20
RANDOM_SEED = 8

random.seed(RANDOM_SEED)

for number_of_the_experiment, _ in enumerate(range(NUMBER_OF_EXPERIMENTS)):
    NUMBER_OF_NODES = random.randint(MIN_NUMBER_OF_NODES, MAX_NUMBER_OF_NODES)
    NUMBER_OF_PATHS = random.randint(MIN_NUMBER_OF_PATHS, MAX_NUMBER_OF_PATHS)
    
    digraph = RandomSESEPathBased(number_of_nodes=NUMBER_OF_NODES, max_path_len=MAX_PATH_LEN, min_path_len=MIN_PATH_LEN, number_of_paths=NUMBER_OF_PATHS).digraph_manager
    tarjan = TarjanSSC(digraph)
    scc_set = tarjan.run()

    graph = get_graph(digraph, NUMBER_OF_NODES)

    other_algorithm_time_start = time.time()
    fpp = findPrimePaths(graph)
    other_algorithm_time_end = time.time()

    ppg = PrimePathsGenerator(digraph)
    reversed_dict = ppg.reversed_translation_dict
    pwc = ppg.get_cycles()
    ppwc = ppg.get_prime_paths_without_cycles()

    time_start = time.time()
    simple_path_res = [x for x in ppwc if x]
    simple_paths_time_end = time.time()
    cycle_res = [x for x in pwc]
    cycles_time_end = time.time()

    NUMBER_OF_SIMPLE_CYCLES = len(cycle_res)
    NUMBER_OF_SIMPLE_PATHS = len(simple_path_res)
    SIMPLE_CYCLE_PROCESSING_TIME = cycles_time_end - simple_paths_time_end
    SIMPLE_PATHS_PROCESSING_TIME = simple_paths_time_end - time_start
    PROCESSING_TIME = cycles_time_end - time_start
    OLD_ALGORITHM_PROCESSING_TIME = other_algorithm_time_end - other_algorithm_time_start 
    NUMBER_OF_NODES = len(digraph.nodes)
    NUMBER_OF_EDGES = len(digraph.edges)
    NUMBER_OF_PRIME_PATHS = len(fpp)
    NUMBER_OF_SCC = len(scc_set)

    exp_result = f"{PROCESSING_TIME=}, {OLD_ALGORITHM_PROCESSING_TIME=}, {NUMBER_OF_SIMPLE_CYCLES=}, {NUMBER_OF_SIMPLE_PATHS=}, {NUMBER_OF_PRIME_PATHS=}, {NUMBER_OF_SCC=}, {NUMBER_OF_EDGES=}, {NUMBER_OF_NODES=}, {SIMPLE_CYCLE_PROCESSING_TIME=}, {SIMPLE_PATHS_PROCESSING_TIME=}\n"
    print(number_of_the_experiment, exp_result)

    with open("./notebooks/prime_paths_exp_results/results.txt", 'a') as f:
        f.write(exp_result)


0 PROCESSING_TIME=0.02519989013671875, OLD_ALGORITHM_PROCESSING_TIME=0.0491180419921875, NUMBER_OF_SIMPLE_CYCLES=220, NUMBER_OF_SIMPLE_PATHS=1763, NUMBER_OF_PRIME_PATHS=3234, NUMBER_OF_SCC=3, NUMBER_OF_EDGES=41, NUMBER_OF_NODES=15, SIMPLE_CYCLE_PROCESSING_TIME=0.002933025360107422, SIMPLE_PATHS_PROCESSING_TIME=0.022266864776611328

1 PROCESSING_TIME=20.239393949508667, OLD_ALGORITHM_PROCESSING_TIME=51.60520386695862, NUMBER_OF_SIMPLE_CYCLES=68660, NUMBER_OF_SIMPLE_PATHS=1347174, NUMBER_OF_PRIME_PATHS=2286739, NUMBER_OF_SCC=3, NUMBER_OF_EDGES=79, NUMBER_OF_NODES=22, SIMPLE_CYCLE_PROCESSING_TIME=0.2862510681152344, SIMPLE_PATHS_PROCESSING_TIME=19.953142881393433

2 PROCESSING_TIME=0.09592700004577637, OLD_ALGORITHM_PROCESSING_TIME=0.23585891723632812, NUMBER_OF_SIMPLE_CYCLES=209, NUMBER_OF_SIMPLE_PATHS=1231, NUMBER_OF_PRIME_PATHS=2886, NUMBER_OF_SCC=3, NUMBER_OF_EDGES=42, NUMBER_OF_NODES=16, SIMPLE_CYCLE_PROCESSING_TIME=0.006345987319946289, SIMPLE_PATHS_PROCESSING_TIME=0.089581012725830

In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt

filename = "./notebooks/prime_paths_exp_results/results.txt"
old_values = []
new_values = []
primes_values = []
edges_values = []
nodes_values = []
scc_values = []

with open(filename, 'r') as f:
    for line in f:
        match = re.search(
            r'PROCESSING_TIME=([\d.]+), OLD_ALGORITHM_PROCESSING_TIME=([\d.]+),.*?NUMBER_OF_PRIME_PATHS=(\d+),.*?NUMBER_OF_SCC=(\d+), NUMBER_OF_EDGES=(\d+), NUMBER_OF_NODES=(\d+)',
            line
        )
        if match:
            new = float(match.group(1))
            old = float(match.group(2))
            primes = int(match.group(3))
            scc = int(match.group(4))
            edges = int(match.group(5))
            nodes = int(match.group(6))

            old_values.append(old)
            new_values.append(new)
            primes_values.append(primes)
            scc_values.append(scc)
            edges_values.append(edges)
            nodes_values.append(nodes)

print("=== OLD Algorithm Stats (Time) ===")
print(f"mean:    {np.mean(old_values):.16f} s")
print(f"median:  {np.median(old_values):.16f} s")
print(f"std:     {np.std(old_values):.16f} s")
print(f"min:     {np.min(old_values):.16f} s")
print(f"max:     {np.max(old_values):.16f} s")
print()

print("=== NEW Algorithm Stats (Time)===")
print(f"mean:    {np.mean(new_values):.16f} s")
print(f"median:  {np.median(new_values):.16f} s")
print(f"std:     {np.std(new_values):.16f} s")
print(f"min:     {np.min(new_values):.16f} s")
print(f"max:     {np.max(new_values):.16f} s")
print()

speedup = [old / new for old, new in zip(old_values, new_values)]

print("=== Speedup (OLD / NEW) ===")
print(f"mean:    {np.mean(speedup):.16f}x")
print(f"median:  {np.median(speedup):.16f}x")
print(f"std:     {np.std(speedup):.16f}x")
print(f"min:     {np.min(speedup):.16f}x")
print(f"max:     {np.max(speedup):.16f}x")
print()

# --- Primes ---
print("=== PRIMES Stats (number)===")
print(f"mean:   {np.mean(primes_values):.16f}")
print(f"median: {np.median(primes_values):.16f}")
print(f"std:    {np.std(primes_values):.16f}")
print(f"min:    {np.min(primes_values)}")
print(f"max:    {np.max(primes_values)}")

print("=== NODES Stats (number)===")
print(f"mean:   {np.mean(nodes_values):.16f}")
print(f"median: {np.median(nodes_values):.16f}")
print(f"std:    {np.std(nodes_values):.16f}")
print(f"min:    {np.min(nodes_values)}")
print(f"max:    {np.max(nodes_values)}")
print()

print("=== EDGES Stats (number)===")
print(f"mean:   {np.mean(edges_values):.16f}")
print(f"median: {np.median(edges_values):.16f}")
print(f"std:    {np.std(edges_values):.16f}")
print(f"min:    {np.min(edges_values)}")
print(f"max:    {np.max(edges_values)}")

print("=== SCC Stats (number)===")
print(f"mean:   {np.mean(scc_values):.16f}")
print(f"median: {np.median(scc_values):.16f}")
print(f"std:    {np.std(scc_values):.16f}")
print(f"min:    {np.min(scc_values)}")
print(f"max:    {np.max(scc_values)}")


=== OLD Algorithm Stats (Time) ===
mean:    6.8970013005101780 s
median:  0.2058260440826416 s
std:     43.6686389234168786 s
min:     0.0001380443572998 s
max:     1073.3611388206481934 s

=== NEW Algorithm Stats (Time)===
mean:    2.9167854007492720 s
median:  0.0767520666122437 s
std:     25.2455227188854927 s
min:     0.0004110336303711 s
max:     625.6487662792205811 s

=== Speedup (OLD / NEW) ===
mean:    2.6193666781250760x
median:  2.6016842488486773x
std:     1.2645951127531043x
min:     0.2288087596752879x
max:     18.9280278241272448x

=== PRIMES Stats (number)===
mean:   258146.1726495726616122
median: 8878.5000000000000000
std:    1472181.1673928159289062
min:    2
max:    31015423
=== NODES Stats (number)===
mean:   18.6829059829059823
median: 18.0000000000000000
std:    4.4621253339002358
min:    8
max:    35

=== EDGES Stats (number)===
mean:   53.4786324786324769
median: 53.0000000000000000
std:    16.2911534331110950
min:    9
max:    107
=== SCC Stats (number)===
mea